# Chinese Remainder Theorem
In this notebook, I try to learn and implement routines for the Chinese Remainder Problem. Specifically, I will try to follow the [Wikipedia article](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) (material copied here for convenience) on the topic. 

**Statement**

Let $n_1, \cdots, n_k$ be integers greater than $1$, which are often called moduli or divisors. Let us denote by $N$ the product of the $n_i$.

The Chinese remainder theorem asserts that if the $n_i$ are pairwise coprime, and if $a_1, \cdots, a_k$ are integers such that $0 ≤ ai < n_i$ for every $i$, then there is one and only one integer $x$, such that $0 \leq x < N$ and the remainder of the Euclidean division of $x$ by $n_i$ is $a_i$ for every $i$.

This may be restated as follows in term of congruences: If the $n_i$ are pairwise coprime, and if $a_1, \cdots, a_k$ are any integers, then there exist integers $x$ such that

$$
\begin{align} 
 x &\equiv a_1  \pmod{n_1} \\
 &\vdots             \\ 
 x &\equiv a_k \pmod{n_k}, \\ 
\end{align}
$$

and any two solutions, say $x_1$ and $x_2$, are congruent modulo $N$, that is, $x_1 ≡ x_2 \pmod{N}$

Example:

$$
\begin{align}
 x &\equiv 0 \pmod 3 \\
 x &\equiv 3 \pmod 4 \\
 x &\equiv 4 \pmod 5.
\end{align}
 $$

**Congruence** is defined as:

For a given positive integer $n$, two integers $a$ and $b$ are called '''congruent modulo $n$''', written: 

$$
a \equiv b \pmod{n}
$$

if $a - b$ is divisible by $n$ (or equivalently if $a$ and $b$ have the same remainder when divided by $n$).


# Systematic search
It is easy to check whether a value of $x$ is a solution: it suffices to compute the remainder of the Euclidean division of $x$ by each $n_i$. Thus, to find the solution, it suffices to check successively the integers from $0$ to $N$ until finding the solution.

In [ ]:
from functools import reduce

# Example -> 39
a_s = [0, 3, 4]
n_s = [3, 4, 5]

# Example -> 301
#a_s = [1, 1, 1, 0]
#n_s = [3, 4, 5, 7] 

N = reduce(lambda x, y: x *y, n_s)

for x_cand in range(0, N + 1):
  remainders_count = 0
  for i, (a, n) in enumerate(zip(a_s, n_s)):
    if(x_cand % n == a):
      remainders_count += 1
    else: break # no need to check more if one fails

  if(remainders_count == len(n_s)):
    print("Found x that satisfies the problem: ", x_cand)
    break


Found x that satisfies the problem:  39


Although very simple, this method is very inefficient. For the simple example considered here, $40$ integers (including $0$) have to be checked for finding the solution, which is $39$. This is an exponential time algorithm, as the size of the input is, up to a constant factor, the number of digits of $n$, and the average number of operations is of the order of $n$.

Therefore, this method is rarely used, neither for hand-written computation nor on computers.

# Search by sieving

The search of the solution may be made dramatically faster by sieving. For this method, we suppose, without loss of generality, that $0\le a_i < n_i$ (if it were not the case, it would suffice to replace each $a_i$ by the remainder of its division by $n_i$). This implies that the solution belongs to the arithmetic progression: 

$$
a_1, a_1 + n_1, a_1+2n_1, \ldots
$$

By testing the values of these numbers modulo $n_2,$ one eventually finds a solution $x_2$ of the two first congruences. Then the solution belongs to the arithmetic progression: 

$$
x_2, x_2 + n_1n_2, x_2+2n_1n_2, \ldots
$$


Testing the values of these numbers modulo $n_3,$, and continuing until every modulus has been tested gives eventually the solution.


This method is faster if the moduli have been ordered by decreasing value, that is if $ n_{1}>n_{2}>\cdots >n_{k}.$ For the example, this gives the following computation. We consider first the numbers that are congruent to 4 modulo 5 (the largest modulus), which are

 4, 9 = 4 + 5, 
 14 = 9 + 5, ... 
 
For each of them, compute the remainder by 4 (the second largest modulus) until getting a number congruent to 3 modulo 4. Then one can proceed by adding 20 = 5×4 at each step, and computing only the remainders by 3. This gives

4 mod 4 → 0. Continue

4 + 5 = 9 mod 4 →1. Continue

9 + 5 = 14 mod 4 → 2. Continue

14 + 5 = 19 mod 4 → 3. OK, continue by considering remainders modulo 3 and adding 5×4 = 20 each time

19 mod 3 → 1. Continue

19 + 20 = 39 mod 3 → 0. OK, this is the result.

This method works well for hand-written computation with a product of moduli that is not too big. However, it is much slower than other methods, for very large products of moduli. Although dramatically faster than the systematic search, this method also has an exponential time complexity and is therefore not used on computers.

In [ ]:
from functools import reduce

# Example -> 39
a_s = [0, 3, 4]
n_s = [3, 4, 5]

# Example -> 301
#a_s = [1, 1, 1, 0]
#n_s = [3, 4, 5, 7]

running_product = 1 
previous_solution = a_s[0]
for k, (a, n) in enumerate(zip(a_s[:-1], n_s[:-1])):
  next_a = a_s[k + 1]
  next_n = n_s[k + 1]

  running_product *= n # Will hold n_1, n_1 * n_2, n_1 * n_2 * n_3, ... for iterations, k = 0, 1, 2...
  
  print('k:', k, 'a:', a, 'n:', n, 'next_n:', next_n, 'running_product', running_product)
  i = 0
  while(True):
    candidate = previous_solution + i * running_product
    remainder = candidate % next_n

    #print('candidate ({}) mod ({}) = remainder ({})'.format(candidate, next_n, remainder))
    if(remainder == next_a):
      previous_solution = candidate
      print('Found', candidate)
      break

    i += 1


k: 0 a: 0 n: 3 next_n: 4 running_product 3
Found 3
k: 1 a: 3 n: 4 next_n: 5 running_product 12
Found 39


# Advent of Code 2020 - Day 13 - Part 2

Now that we have implemented an efficient method (compared to brute-force search) for computing a solution, we can check if we can use CRT to solve this problem.



In [ ]:
f = open("inputs_day_13.txt", "r")
inputs_day_13 = []
for line in f:
  line_formatted = line.strip()
  #print(laine_formatted)
  inputs_day_13.append(line_formatted)
print(inputs_day_13)
print(len(inputs_day_13))

['1002461', '29,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,521,x,x,x,x,x,x,x,23,x,x,x,x,13,x,x,x,17,x,x,x,x,x,x,x,x,x,x,x,x,x,601,x,x,x,x,x,37,x,x,x,x,x,x,x,x,x,x,x,x,19']
2


In [ ]:
bus_ids_all = [bus_id for bus_id in inputs_day_13[1].split(',')]
offsets = {}
for i, bus_id in enumerate(bus_ids_all):
  if(bus_id != 'x'):
    offsets[int(bus_id)] = i
print('offsets', offsets)
bus_ids = [int(bus_id) for bus_id in bus_ids_all if bus_id != 'x']
#bus_ids.sort(reverse=True)
indexes = [offsets[bus_id] for bus_id in bus_ids]
print('bus_ids', bus_ids)
print('indexes', indexes)

offsets {29: 0, 41: 19, 521: 29, 23: 37, 13: 42, 17: 46, 601: 60, 37: 66, 19: 79}
bus_ids [29, 41, 521, 23, 13, 17, 601, 37, 19]
indexes [0, 19, 29, 37, 42, 46, 60, 66, 79]


Looking at the above, we can write a system of equations:


$$
(t + 0) \mod 29 = 0 \\
(t + 19) \mod 41 = 0 \\
(t + 29) \mod 521 = 0 \\
(t + 37) \mod 13 = 0 \\
(t + 42) \mod 17 = 0 \\
(t + 46) \mod 37 = 0 \\
(t + 60) \mod 601 = 0 \\
(t + 66) \mod 37 = 0 \\
(t + 79) \mod 19 = 0 \\
$$

**(Cheating)**

An equation solver such as Wolfram Alpha is able to [solve](https://www.wolframalpha.com/input/?i=solve+%28t+%2B+0%29+mod+29+%3D+0%3B+%28t+%2B+19%29+mod+41+%3D+0%3B+%28t+%2B+29%29+mod+521+%3D+0%3B+%28t+%2B+37%29+mod+23+%3D+0%3B+%28t+%2B+42%29+mod+13+%3D+0%3B+%28t+%2B+46%29+mod+17+%3D+0%3B+%28t+%2B+60%29+mod+601+%3D+0%3B+%28t+%2B+66%29+mod+37%3D+0%3B%28t+%2B+79%29+mod+19+%3D+0%3B) this problem:

$$
t = 1330360937940281 n + 725850285300475, n \in \mathbb{Z}
$$

**Using CRT**

Could we cast the problem as system of congruences?

Consulting the definition of congruence, we can get:

$$
(a-b) \mod n = 0
$$

This is (almost) in the same format as the CRT statement.

$$
t \equiv -0 \pmod{29} \\
t \equiv -19 \pmod{41}\\
t \equiv -29 \pmod{521}\\
t \equiv -37 \pmod{13}\\
t \equiv -42 \pmod{17} \\
t \equiv -46 \pmod{37} \\
t \equiv -60 \pmod{601} \\
t \equiv -66 \pmod{37} \\
t \equiv -79 \pmod{19} \\
$$

It does not exactly match the CRT statement that all $a_k$ be positive. However, we can find positive integers that give the same remainders. We replace a negative value $a_k$ with $a_k \mod n_k$.


$$
t \equiv 0 \pmod{29} \\
t \equiv 22 \pmod{41}\\
t \equiv 492 \pmod{521}\\
t \equiv 521 \pmod{13}\\
t \equiv 23 \pmod{17} \\
t \equiv 13 \pmod{37} \\
t \equiv 17 \pmod{601} \\
t \equiv 601 \pmod{37} \\
t \equiv 37 \pmod{19} \\
$$



In [ ]:
a_s = [-index % bus_id for index, bus_id in zip(indexes, bus_ids)]
n_s = bus_ids
print('a_s', a_s)
print('n_s', n_s)

a_s [0, 22, 492, 9, 10, 5, 541, 8, 16]
n_s [29, 41, 521, 23, 13, 17, 601, 37, 19]


In [ ]:
import time
from functools import reduce

start_time = time.time()

running_product = 1 
previous_solution = a_s[0]
count = 0
for k, (a, n) in enumerate(zip(a_s[:-1], n_s[:-1])):
  next_a = a_s[k + 1]
  next_n = n_s[k + 1]

  running_product *= n # Will hold n_1, n_1 * n_2, n_1 * n_2 * n_3, ... for iterations, k = 0, 1, 2...
  
  print('k:', k, 'a:', a, 'n:', n, 'next_n:', next_n, 'running_product', running_product)
  i = 0
  while(True):
    candidate = previous_solution + i * running_product
    remainder = candidate % next_n
    count += 1
    #print('candidate ({}) mod ({}) = remainder ({})'.format(candidate, next_n, remainder))
    if(remainder == next_a):
      previous_solution = candidate
      print('Found', candidate)
      break

    i += 1

print("\nExecution completed in {}s seconds.".format(time.time() - start_time))

k: 0 a: 0 n: 29 next_n: 41 running_product 29
Found 145
k: 1 a: 22 n: 41 next_n: 521 running_product 1189
Found 302151
k: 2 a: 492 n: 521 next_n: 23 running_product 619469
Found 10833124
k: 3 a: 9 n: 23 next_n: 13 running_product 14247787
Found 139063207
k: 4 a: 10 n: 13 next_n: 17 running_product 185221231
Found 1065169362
k: 5 a: 5 n: 17 next_n: 601 running_product 3148760927
Found 1059048840834
k: 6 a: 541 n: 601 next_n: 37 running_product 1892405317127
Found 25660317963485
k: 7 a: 8 n: 37 next_n: 19 running_product 70018996733699
Found 725850285300475

Execution completed in 0.0058286190032958984s seconds.


We get the correct answer $725850285300475$ (725 trillion 850 billion 285 million 300 thousand 475), which we found with Wolfram Alpha. The search methods were easy enough to implement, but I need to study the CRT more carefully. I think it may be a good gateway into number theory.